Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [7]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
print("train_labels[0]--",train_labels[0])

train_labels[0]-- [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]


In [6]:
print(np.arange(num_labels))

[0 1 2 3 4 5 6 7 8 9]


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [16]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    print('data---',data.get_shape())
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    print('layer1_weights---',layer1_weights.get_shape())
    print('conv2d(data, layer1_weights---',conv.get_shape())
    hidden = tf.nn.relu(conv + layer1_biases)
    
    print('relu(conv + layer1_biases)---',hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    print('layer2_weights---',layer2_weights.get_shape())
    print('conv2d(hidden, layer2_weights---',conv.get_shape())
    
    hidden = tf.nn.relu(conv + layer2_biases)
    print('relu(conv + layer2_biases)---',hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])---',reshape.get_shape())
    print('layer3_weights---',layer3_weights.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('relu(tf.matmul(reshape, layer3_weights) + layer3_biases)---',hidden.get_shape())
    print('layer4_weights---',layer4_weights.get_shape())
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  print('logits--',logits.get_shape())
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

data--- (16, 28, 28, 1)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (16, 14, 14, 16)
relu(conv + layer1_biases)--- (16, 14, 14, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (16, 7, 7, 16)
relu(conv + layer2_biases)--- (16, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])--- (16, 784)
layer3_weights--- (784, 64)
relu(tf.matmul(reshape, layer3_weights) + layer3_biases)--- (16, 64)
layer4_weights--- (64, 10)
logits-- (16, 10)
data--- (10000, 28, 28, 1)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (10000, 14, 14, 16)
relu(conv + layer1_biases)--- (10000, 14, 14, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (10000, 7, 7, 16)
relu(conv + layer2_biases)--- (10000, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])--- (10000, 784)
layer3_weights--- (784, 64)
relu(tf.matmul(reshape, layer3_weights) + layer3_biases)--- (10000, 64)
layer4_weights--- (64, 10)
data--- (1000

In [17]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.277282
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.463182
Minibatch accuracy: 31.2%
Validation accuracy: 58.5%
Minibatch loss at step 100: 0.886016
Minibatch accuracy: 68.8%
Validation accuracy: 73.8%
Minibatch loss at step 150: 0.426313
Minibatch accuracy: 81.2%
Validation accuracy: 75.4%
Minibatch loss at step 200: 0.821821
Minibatch accuracy: 75.0%
Validation accuracy: 78.4%
Minibatch loss at step 250: 1.111218
Minibatch accuracy: 68.8%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.382894
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 350: 0.471884
Minibatch accuracy: 93.8%
Validation accuracy: 77.4%
Minibatch loss at step 400: 0.192867
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.888107
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.543746
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    print('data---',data.get_shape())
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer1_weights---',layer1_weights.get_shape())
    print('conv2d(data, layer1_weights---',conv.get_shape())
    hidden = tf.nn.relu(sub + layer1_biases)
    
    print('relu(conv + layer1_biases)---',hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer2_weights---',layer2_weights.get_shape())
    print('conv2d(hidden, layer2_weights---',conv.get_shape())
    
    hidden = tf.nn.relu(sub + layer2_biases)
    print('relu(conv + layer2_biases)---',hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])---',reshape.get_shape())
    print('layer3_weights---',layer3_weights.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('relu(tf.matmul(reshape, layer3_weights) + layer3_biases)---',hidden.get_shape())
    print('layer4_weights---',layer4_weights.get_shape())
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  print('logits--',logits.get_shape())
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

data--- (16, 28, 28, 1)
max_pool([1,2,2,1]--- (16, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (16, 28, 28, 16)
relu(conv + layer1_biases)--- (16, 14, 14, 16)
max_pool([1,2,2,1]--- (16, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (16, 14, 14, 16)
relu(conv + layer2_biases)--- (16, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])--- (16, 784)
layer3_weights--- (784, 64)
relu(tf.matmul(reshape, layer3_weights) + layer3_biases)--- (16, 64)
layer4_weights--- (64, 10)
logits-- (16, 10)
data--- (10000, 28, 28, 1)
max_pool([1,2,2,1]--- (10000, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (10000, 28, 28, 16)
relu(conv + layer1_biases)--- (10000, 14, 14, 16)
max_pool([1,2,2,1]--- (10000, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (10000, 14, 14, 16)
relu(conv + layer2_biases)--- (10000, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape

In [20]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.982834
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 50: 2.314407
Minibatch accuracy: 6.2%
Validation accuracy: 18.1%
Minibatch loss at step 100: 1.798905
Minibatch accuracy: 31.2%
Validation accuracy: 41.8%
Minibatch loss at step 150: 0.738172
Minibatch accuracy: 87.5%
Validation accuracy: 68.0%
Minibatch loss at step 200: 1.165806
Minibatch accuracy: 68.8%
Validation accuracy: 73.8%
Minibatch loss at step 250: 1.279483
Minibatch accuracy: 68.8%
Validation accuracy: 74.2%
Minibatch loss at step 300: 0.510097
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.399438
Minibatch accuracy: 87.5%
Validation accuracy: 76.9%
Minibatch loss at step 400: 0.209458
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
Minibatch loss at step 450: 1.109126
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.706545
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Mini

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [24]:
#Adding hidden = tf.nn.dropout(hidden, keep_prob)


batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    print('data---',data.get_shape())
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer1_weights---',layer1_weights.get_shape())
    print('conv2d(data, layer1_weights---',conv.get_shape())
    hidden = tf.nn.relu(sub + layer1_biases)
    hidden = tf.nn.dropout(hidden, keep_prob)

    
    
    print('dropout after relu(conv + layer1_biases)---',hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer2_weights---',layer2_weights.get_shape())
    print('conv2d(hidden, layer2_weights---',conv.get_shape())
    
    hidden = tf.nn.relu(sub + layer2_biases)
    hidden = tf.nn.dropout(hidden, keep_prob)

    print('dropout after relu(conv + layer2_biases)---',hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])---',reshape.get_shape())
    print('layer3_weights---',layer3_weights.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('relu(tf.matmul(reshape, layer3_weights) + layer3_biases)---',hidden.get_shape())
    print('layer4_weights---',layer4_weights.get_shape())
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset,0.9375)
  print('logits--',logits.get_shape())
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset,0.80))
  test_prediction = tf.nn.softmax(model(tf_test_dataset,0.80))

data--- (16, 28, 28, 1)
max_pool([1,2,2,1]--- (16, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (16, 28, 28, 16)
dropout after relu(conv + layer1_biases)--- (16, 14, 14, 16)
max_pool([1,2,2,1]--- (16, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (16, 14, 14, 16)
dropout after relu(conv + layer2_biases)--- (16, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])--- (16, 784)
layer3_weights--- (784, 64)
relu(tf.matmul(reshape, layer3_weights) + layer3_biases)--- (16, 64)
layer4_weights--- (64, 10)
logits-- (16, 10)
data--- (10000, 28, 28, 1)
max_pool([1,2,2,1]--- (10000, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (10000, 28, 28, 16)
dropout after relu(conv + layer1_biases)--- (10000, 14, 14, 16)
max_pool([1,2,2,1]--- (10000, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (10000, 14, 14, 16)
dropout after relu(conv + layer2_biases)--- (10000, 7, 7, 16

In [25]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.659868
Minibatch accuracy: 12.5%
Validation accuracy: 11.4%
Minibatch loss at step 50: 1.964210
Minibatch accuracy: 25.0%
Validation accuracy: 27.3%
Minibatch loss at step 100: 1.384963
Minibatch accuracy: 56.2%
Validation accuracy: 53.2%
Minibatch loss at step 150: 0.624118
Minibatch accuracy: 81.2%
Validation accuracy: 66.8%
Minibatch loss at step 200: 0.918198
Minibatch accuracy: 75.0%
Validation accuracy: 73.3%
Minibatch loss at step 250: 1.040229
Minibatch accuracy: 75.0%
Validation accuracy: 73.1%
Minibatch loss at step 300: 0.387863
Minibatch accuracy: 87.5%
Validation accuracy: 76.3%
Minibatch loss at step 350: 0.378950
Minibatch accuracy: 93.8%
Validation accuracy: 74.5%
Minibatch loss at step 400: 0.273012
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 450: 0.819191
Minibatch accuracy: 81.2%
Validation accuracy: 76.9%
Minibatch loss at step 500: 0.736919
Minibatch accuracy: 87.5%
Validation accuracy: 77.9%


In [28]:
#adding learning rate




batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    print('data---',data.get_shape())
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer1_weights---',layer1_weights.get_shape())
    print('conv2d(data, layer1_weights---',conv.get_shape())
    hidden = tf.nn.relu(sub + layer1_biases)
    
    print('relu(conv + layer1_biases)---',hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer2_weights---',layer2_weights.get_shape())
    print('conv2d(hidden, layer2_weights---',conv.get_shape())
    
    hidden = tf.nn.relu(sub + layer2_biases)
    print('relu(conv + layer2_biases)---',hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])---',reshape.get_shape())
    print('layer3_weights---',layer3_weights.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('relu(tf.matmul(reshape, layer3_weights) + layer3_biases)---',hidden.get_shape())
    print('layer4_weights---',layer4_weights.get_shape())
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  print('logits--',logits.get_shape())
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  #optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

data--- (16, 28, 28, 1)
max_pool([1,2,2,1]--- (16, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (16, 28, 28, 16)
relu(conv + layer1_biases)--- (16, 14, 14, 16)
max_pool([1,2,2,1]--- (16, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (16, 14, 14, 16)
relu(conv + layer2_biases)--- (16, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])--- (16, 784)
layer3_weights--- (784, 64)
relu(tf.matmul(reshape, layer3_weights) + layer3_biases)--- (16, 64)
layer4_weights--- (64, 10)
logits-- (16, 10)
data--- (10000, 28, 28, 1)
max_pool([1,2,2,1]--- (10000, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (10000, 28, 28, 16)
relu(conv + layer1_biases)--- (10000, 14, 14, 16)
max_pool([1,2,2,1]--- (10000, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (10000, 14, 14, 16)
relu(conv + layer2_biases)--- (10000, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape

In [29]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.911339
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.557084
Minibatch accuracy: 50.0%
Validation accuracy: 55.9%
Minibatch loss at step 100: 0.886877
Minibatch accuracy: 62.5%
Validation accuracy: 69.9%
Minibatch loss at step 150: 0.389921
Minibatch accuracy: 87.5%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.837481
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 250: 1.002365
Minibatch accuracy: 68.8%
Validation accuracy: 78.6%
Minibatch loss at step 300: 0.363925
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 350: 0.428382
Minibatch accuracy: 93.8%
Validation accuracy: 80.4%
Minibatch loss at step 400: 0.210837
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 450: 0.907900
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 500: 0.565959
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%


In [30]:
#adding learning rate




batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    print('data---',data.get_shape())
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer1_weights---',layer1_weights.get_shape())
    print('conv2d(data, layer1_weights---',conv.get_shape())
    hidden = tf.nn.relu(sub + layer1_biases)
    
    print('relu(conv + layer1_biases)---',hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    sub = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], 'SAME')
    print('max_pool([1,2,2,1]---', sub.get_shape())
    print('layer2_weights---',layer2_weights.get_shape())
    print('conv2d(hidden, layer2_weights---',conv.get_shape())
    
    hidden = tf.nn.relu(sub + layer2_biases)
    print('relu(conv + layer2_biases)---',hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])---',reshape.get_shape())
    print('layer3_weights---',layer3_weights.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('relu(tf.matmul(reshape, layer3_weights) + layer3_biases)---',hidden.get_shape())
    print('layer4_weights---',layer4_weights.get_shape())
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  print('logits--',logits.get_shape())
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.95)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

data--- (16, 28, 28, 1)
max_pool([1,2,2,1]--- (16, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (16, 28, 28, 16)
relu(conv + layer1_biases)--- (16, 14, 14, 16)
max_pool([1,2,2,1]--- (16, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (16, 14, 14, 16)
relu(conv + layer2_biases)--- (16, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])--- (16, 784)
layer3_weights--- (784, 64)
relu(tf.matmul(reshape, layer3_weights) + layer3_biases)--- (16, 64)
layer4_weights--- (64, 10)
logits-- (16, 10)
data--- (10000, 28, 28, 1)
max_pool([1,2,2,1]--- (10000, 14, 14, 16)
layer1_weights--- (5, 5, 1, 16)
conv2d(data, layer1_weights--- (10000, 28, 28, 16)
relu(conv + layer1_biases)--- (10000, 14, 14, 16)
max_pool([1,2,2,1]--- (10000, 7, 7, 16)
layer2_weights--- (5, 5, 16, 16)
conv2d(hidden, layer2_weights--- (10000, 14, 14, 16)
relu(conv + layer2_biases)--- (10000, 7, 7, 16)
reshape(hidden, [shape[0], shape[1] * shape[2] * shape

In [31]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.593381
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 50: 1.092840
Minibatch accuracy: 56.2%
Validation accuracy: 65.7%
Minibatch loss at step 100: 0.704389
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 150: 0.414882
Minibatch accuracy: 87.5%
Validation accuracy: 78.7%
Minibatch loss at step 200: 0.859446
Minibatch accuracy: 75.0%
Validation accuracy: 81.1%
Minibatch loss at step 250: 1.074378
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 300: 0.271579
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 350: 0.405483
Minibatch accuracy: 93.8%
Validation accuracy: 82.1%
Minibatch loss at step 400: 0.188220
Minibatch accuracy: 100.0%
Validation accuracy: 83.4%
Minibatch loss at step 450: 1.067577
Minibatch accuracy: 81.2%
Validation accuracy: 83.1%
Minibatch loss at step 500: 0.486476
Minibatch accuracy: 87.5%
Validation accuracy: 83.9%
M